In [ ]:
import requests

def get_response(prompt, history=[], temperature=0.5):
    json = {"prompt": prompt, "history": history, "temperature": temperature}
    response = requests.post(url='http://127.0.0.1:7788/chat', json=json)
    return response.json()['answer']

print(get_response('你是谁？我是谁？'))

In [ ]:
history = []
while True:
    print('*'*50)
    prompt = input()
    if prompt == "q":
        break
    print("ME: " + prompt)
    answer = get_response(prompt, history)
    history.append([prompt, answer])
    print("AI: " + answer)

In [ ]:
# tts
import io
import zipfile
import requests

params_refine_text = {
    "prompt": "", "top_P": 0.7, "top_K": 10, "temperature": 0.3,
    "min_new_token": 0, "max_new_token": 384,
    "stream_batch": 24, "repetition_penalty": 1,
    "ensure_non_empty": True, "show_tqdm": True,
}
params_infer_code = {
    "prompt": "[speed_5]", "top_P": 0.7, "top_K": 10, "temperature": 0.3,
    "min_new_token": 0, "max_new_token": 2048,
    "stream_batch": True, "repetition_penalty": 1.05,
    "ensure_non_empty": True, "show_tqdm": True, "spk_emb": None,
}
json = {
    "text": ["《三体》是一部描绘人类首次接触外星文明并引发一系列宇宙级冲突的科幻小说，讲述了在文化大革命时期，天文学家叶文洁向宇宙发送信息，意外联系到因不稳定恒星系统而寻求新栖息地的三体文明，随后地球人类面临三体文明的入侵威胁，社会分裂成合作派和抵抗派，展开了一场关乎人类命运的斗争，同时探索了科学、文明和宇宙的深层问题。"],
    "audio_seed": 37213721, "text_seed": 37213721,
    "lang": None, "stream": False, "use_decoder": True,
    "skip_refine_text": True, "refine_text_only": False,
    "do_text_normalization": True, "do_homophone_replacement": False,
}
json["params_refine_text"] = params_refine_text
json["params_infer_code"] = params_infer_code

response = requests.post(url="http://127.0.0.1:7788/tts", json=json)
with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_f:
    zip_f.extractall("output/test/")

In [ ]:
# spk_emb
import requests
def sample_random_speaker(audio_seed):
    json = {"seed": audio_seed}
    response = requests.post(url='http://127.0.0.1:7778/tts/srs', json=json)
    return response.json()
sample_random_speaker(1234)

In [ ]:
# dvae_coef
import requests
response = requests.post(url='http://127.0.0.1:7788/tts/reload')
response.json()

In [ ]:
# refine text
import requests
def refine_text(text, text_seed, text_refine):
    if not text_refine:
        return text
    params_refine_text = {"temperature": 0.3, "top_P": 0.7, "top_K": 10}
    json = {"text": text, "text_seed": text_seed, "params_refine_text": params_refine_text}
    response = requests.post(url='http://127.0.0.1:7788/tts/refine', json=json)
    return response.json()
text=refine_text(text="你是谁",text_seed=2222,text_refine=True)
text